In [2]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=1733ac158334cbd362727cfcce65beaf8fc4248434a6447be0580d0be6f2a0e0
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=08ce2ea754ead71735066cd78f1f4df4088126aad8cef17c186c4f17ab9ee96c
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_lables),(test_images,test_lables)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [31]:
def build_model(hp):
  model = keras. Sequential([ keras.layers.Conv2D(
    filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
    kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
    activation='relu',
    input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
                    filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16), 
                    kernel_size=hp.Choice('conv_2_kernel', values = [3,5]), activation='relu'
                ),
    keras.layers. Flatten(),
    keras.layers.Dense( units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16), 
                                    activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax') 
    ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', 
                                                            values=[1e-2, 1e-3])),
                                                            loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

In [25]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [34]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials = 5,
                            directory='output',
                            project_name="Mnist Fashion")

In [35]:
tuner_search.search(train_images,train_lables,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 11m 22s]
val_accuracy: 0.9146666526794434

Best val_accuracy So Far: 0.9146666526794434
Total elapsed time: 01h 39m 59s
INFO:tensorflow:Oracle triggered exit


In [36]:
model=tuner_search.get_best_models(num_models=1)[0]

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2654304   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,683,610
Trainable params: 2,683,610
Non-trainable params: 0
_________________________________________________________________
